In [1]:
from cloudvolume import CloudVolume
import numpy as np
import os
import random
import requests
import tifffile
import matplotlib.pyplot as plt
import fsspec
import zarr
import dask.array as da
import pandas as pd

### Mouse nucleus accumbens - OpenOrganelle

In [2]:
store = fsspec.get_mapper('s3://janelia-cosem-datasets/jrc_mus-nacc-2/jrc_mus-nacc-2.zarr', anon=True)
group = zarr.open(store, mode='r')

In [3]:
print(group.tree())

/
 └── recon-2
     ├── em
     │   └── fibsem-int16
     │       ├── s0 (564, 2520, 2596) int16
     │       ├── s1 (282, 1260, 1298) int16
     │       ├── s2 (141, 630, 649) int16
     │       ├── s3 (70, 315, 324) int16
     │       ├── s4 (35, 157, 162) int16
     │       ├── s5 (17, 78, 81) int16
     │       ├── s6 (8, 39, 40) int16
     │       ├── s7 (4, 19, 20) int16
     │       └── s8 (2, 9, 10) int16
     └── labels
         └── inference
             └── segmentations
                 └── nuc
                     ├── s0 (70, 315, 324) uint8
                     ├── s1 (35, 157, 162) uint8
                     └── s2 (17, 78, 81) uint8


In [4]:
print(list(group.group_keys()))

['recon-2']


In [5]:
zdata = group['recon-2/em/fibsem-int16/s0']

In [6]:
print(zdata.shape)

(564, 2520, 2596)


In [7]:
ddata = da.from_array(zdata, chunks=zdata.chunks)

In [8]:
ddata

dask.array<array, shape=(564, 2520, 2596), dtype=int16, chunksize=(64, 64, 64), chunktype=numpy.ndarray>

In [9]:
ddata_np = ddata.compute()

In [10]:
np.save('Data/fibsem_fullres_s0.npy', ddata_np)

In [11]:
fibsem_meta = group['recon-2']['em']['fibsem-int16'].attrs.asdict()

In [12]:
labels_meta = group['recon-2']['labels']['inference']['segmentations']['nuc'].attrs.asdict()

In [13]:
labels_meta['multiscales'][0]['datasets']

[{'coordinateTransformations': [{'scale': [23.68, 32.0, 32.0],
    'type': 'scale'},
   {'translation': [10.36, 14.0, 14.0], 'type': 'translation'}],
  'path': 's0'},
 {'coordinateTransformations': [{'scale': [47.36, 64.0, 64.0],
    'type': 'scale'},
   {'translation': [22.2, 30.0, 30.0], 'type': 'translation'}],
  'path': 's1'},
 {'coordinateTransformations': [{'scale': [94.72, 128.0, 128.0],
    'type': 'scale'},
   {'translation': [45.879999999999995, 62.0, 62.0], 'type': 'translation'}],
  'path': 's2'}]

In [14]:
fibsem_meta['multiscales'][0]['datasets']

[{'coordinateTransformations': [{'scale': [2.96, 4.0, 4.0], 'type': 'scale'},
   {'translation': [0.0, 0.0, 0.0], 'type': 'translation'}],
  'path': 's0'},
 {'coordinateTransformations': [{'scale': [5.92, 8.0, 8.0], 'type': 'scale'},
   {'translation': [1.48, 2.0, 2.0], 'type': 'translation'}],
  'path': 's1'},
 {'coordinateTransformations': [{'scale': [11.840000000000002, 16.0, 16.0],
    'type': 'scale'},
   {'translation': [4.4399999999999995, 6.0, 6.0], 'type': 'translation'}],
  'path': 's2'},
 {'coordinateTransformations': [{'scale': [23.68, 32.0, 32.0],
    'type': 'scale'},
   {'translation': [10.36, 14.0, 14.0], 'type': 'translation'}],
  'path': 's3'},
 {'coordinateTransformations': [{'scale': [47.36, 64.0, 64.0],
    'type': 'scale'},
   {'translation': [22.2, 30.0, 30.0], 'type': 'translation'}],
  'path': 's4'},
 {'coordinateTransformations': [{'scale': [94.72, 128.0, 128.0],
    'type': 'scale'},
   {'translation': [45.88, 62.0, 62.0], 'type': 'translation'}],
  'path': '

In [ ]:
fibsem_meta_rows = []
for ds in fibsem_meta['multiscales'][0]['datasets']:
    path = ds['path']
    scale = translation = None
    for tf in ds['coordinateTransformations']:
        if tf['type'] == 'scale':
            scale = tf['scale']
        if tf['type'] == 'translation':
            translation = tf['translation']
    fibsem_meta_rows.append({
        'Resolution': path,
        'Voxel size (z, y, x)[nm]': scale,
        'Translation (z, y, x)[nm]': translation
    })

fibsem_meta_df = pd.DataFrame(fibsem_meta_rows)
fibsem_meta_df.to_csv('Open_Organelle/fibsem_meta.csv', index=False)

In [16]:
print(fibsem_meta_df.to_markdown(index=False))

| Resolution   | Voxel size (z, y, x)[nm]            | Translation (z, y, x)[nm]      |
|:-------------|:------------------------------------|:-------------------------------|
| s0           | [2.96, 4.0, 4.0]                    | [0.0, 0.0, 0.0]                |
| s1           | [5.92, 8.0, 8.0]                    | [1.48, 2.0, 2.0]               |
| s2           | [11.840000000000002, 16.0, 16.0]    | [4.4399999999999995, 6.0, 6.0] |
| s3           | [23.68, 32.0, 32.0]                 | [10.36, 14.0, 14.0]            |
| s4           | [47.36, 64.0, 64.0]                 | [22.2, 30.0, 30.0]             |
| s5           | [94.72, 128.0, 128.0]               | [45.88, 62.0, 62.0]            |
| s6           | [189.44, 256.0, 256.0]              | [93.24, 126.0, 126.0]          |
| s7           | [378.8799999999999, 512.0, 512.0]   | [187.96, 254.0, 254.0]         |
| s8           | [757.7599999999999, 1024.0, 1024.0] | [377.4, 510.0, 510.0]          |


In [ ]:
labels_meta_rows = []
for ds in labels_meta['multiscales'][0]['datasets']:
    path = ds['path']
    scale = translation = None
    for tf in ds['coordinateTransformations']:
        if tf['type'] == 'scale':
            scale = tf['scale']
        if tf['type'] == 'translation':
            translation = tf['translation']
    labels_meta_rows.append({
        'Resolution': path,
        'Voxel size (z, y, x)[nm]': scale,
        'Translation (z, y, x)[nm]': translation
    })
labels_meta_df = pd.DataFrame(labels_meta_rows)
labels_meta_df.to_csv('Open_Organelle/fibsem_labels_meta.csv', index=False)

In [18]:
print(labels_meta_df.to_markdown(index=False))

| Resolution   | Voxel size (z, y, x)[nm]   | Translation (z, y, x)[nm]        |
|:-------------|:---------------------------|:---------------------------------|
| s0           | [23.68, 32.0, 32.0]        | [10.36, 14.0, 14.0]              |
| s1           | [47.36, 64.0, 64.0]        | [22.2, 30.0, 30.0]               |
| s2           | [94.72, 128.0, 128.0]      | [45.879999999999995, 62.0, 62.0] |
